In [10]:
# import libraries
from datetime import date, datetime   # Convert date string to datetime format and back

In [2]:
fileName = "data.csv"

In [3]:
# Return: validLines, numberLines
openDict = {}    # Dictionary that contains <processID, lineNumber> for a start process.
validLines = []  # Lines numbers of valid lines in file


with open(fileName, 'r') as readFile:
    lineNumber = 0
    for line in readFile:
        # ------------------------------------------------------------
        # START line found => add {processID: lineNumber} to openDict
        # ------------------------------------------------------------
        if line.startswith('$START'):
            # Get process ID
            processID = line.split(',')[1]
            # Add {processID: lineNumber} entry to openDict
            openDict[processID] = lineNumber

        # ------------------------------------------------------    
        # END line found:
        #     - Check if start entry for processID exists
        #     - Add line of START and END entry to 'validLines'
        #     - Delete START entry from dict openDict
        # ------------------------------------------------------
        elif line.startswith("$END"):
            # Check if process ID for END process has a corresponding start time
            # i.e. check if processID entry exists in openDict
            processID = line.split(',')[1]
            if processID in openDict:
                validLines.append(openDict[processID])
                validLines.append(lineNumber)
                del openDict[processID]
            
        lineNumber = lineNumber + 1

numberLines = lineNumber;
print("Total number of lines = " + str(lineNumber))
print("Valid lines = " + str(len(validLines)))
print("Non-terminated processes = " + str(len(openDict)))

Total number of lines = 5
Valid lines = 4
Non-terminated processes = 1


In [34]:
def determineValidEntries( fileName ):
    print("Check for valid entries in log file " + str(fileName) + "...")
      # Return: validLines, numberLines
    openDict = {}    # Dictionary that contains <processID, lineNumber> for a start process.
    validLines = []  # Lines numbers of valid lines in file


    with open(fileName, 'r') as readFile:
        lineNumber = 0
        for line in readFile:
            # ------------------------------------------------------------
            # START line found => add {processID: lineNumber} to openDict
            # ------------------------------------------------------------
            if line.startswith('$START'):
                # Get process ID
                processID = line.split(',')[1]
                # Add {processID: lineNumber} entry to openDict
                openDict[processID] = lineNumber

            # ------------------------------------------------------    
            # END line found:
            #     - Check if start entry for processID exists
            #     - Add line of START and END entry to 'validLines'
            #     - Delete START entry from dict openDict
            # ------------------------------------------------------
            elif line.startswith("$END"):
                # Check if process ID for END process has a corresponding start time
                # i.e. check if processID entry exists in openDict
                processID = line.split(',')[1]
                if processID in openDict:
                    validLines.append(openDict[processID])
                    validLines.append(lineNumber)
                    del openDict[processID]

            lineNumber = lineNumber + 1

    numberLines = lineNumber;
    
    print("\tTotal number of lines = " + str(lineNumber))
    print("\tValid lines = " + str(len(validLines)))
    print("\tNon-terminated processes = " + str(len(openDict))) 
    
    return validLines, numberLines

In [35]:
validLines, numberLines = determineValidEntries(fileName)

Check for valid entries in log file data.csv...
	Total number of lines = 5
	Valid lines = 4
	Non-terminated processes = 1


In [29]:
lineValid = [False] * numberLines
for idx in validLines:
    lineValid[idx] = True
    
noInstances = 0      # Counts how many instances are open at the same time    
timeMap = {}      # {ProcessID: time}
timeVector = []
toolboxes = []

with open(fileName, 'r') as readFile:
    lineNumber = 0
    for line in readFile:
        # 'non valid' lines are ignored
        if lineValid[lineNumber]:
            
            # ------------------------------------------------------------
            # START line found:
            #     - add {processID: start time} to timeMap dict.
            # ------------------------------------------------------------
            if line.startswith('$START'):
                line = line.rstrip('\n')  # remove \n at end of line
                
                # Get process ID and time stamp
                substr = line.split(',')  # split in substrings
                processID = substr[1]     # get process ID
                timeStamp = substr[4]     # get time stamp and convert to datetime format
                timeStamp = datetime.strptime(timeStamp, "%d.%m.%Y %H:%M")
                
                # Add process ID and time stamp to dict
                timeMap[processID] = timeStamp
                
                # Increase number of active instances
                noInstances = noInstances + 1
        
            # -----------------------------------------------------
            # END line found:
            # If the last active instance:
            #    - Process time stamps and write output data
            # else
            #    - Add [start, end time] of process to timeVector
            #    - Add toolboxes of process to toolboxes
            #    Interpretation:
            #    timeVector[2n], timeVector[2n+1] is the start/end time
            #    of a process with toolboxes toolboxes[n].
            # -----------------------------------------------------
        
            elif line.startswith("$END"):
                
                # Get data
                line = line.rstrip('\n')  # remove \n at end of line
                
                substr = line.split(',')  # split in substrings
                
                processID = substr[1]
                licenseNo = substr[2]
                version   = substr[3]
                endTime = substr[4]
                endTime = datetime.strptime(endTime, "%d.%m.%Y %H:%M")
                tbox = substr[5].strip('][').split(':')
                print(tbox)
                
                # Add start - end time and toolboxes
                if not processID in timeMap:
                    print("ERROR: (line number = " + str(lineNumber) + ")" )
                    print("No start time found for the end timestamp.")
                    #return
                else:
                    startTime = timeMap[processID]
                    timeVector.append(startTime)
                    timeVector.append(endTime)
                     
                    toolboxes.append(tbox)
                
                # One instance closed by "END" => decrease active instance counter
                noInstances = noInstances - 1
                
                # ---------------------------------------------------
                # If was last active instance: process & output data
                # ---------------------------------------------------
                if noInstances == 0:
                    toolboxes, timeVectors = processTime(timeVector, toolboxes)
                       
                    for cc in range(len(toolboxes)):
                        toolbox = toolboxes[cc]
                        noIntervals = int(len(timeVectors[cc])/2)
                        
                        for cy in range(noIntervals):
                            startTime = timeVectors[cc][2*cy]
                            endTime = timeVectors[cc][2*cy+1]
                            
                            print(toolboxes[cc] + ": "
                                  + startTime.strftime("%d.%m.%Y %H:%M") + ", "
                                  + endTime.strftime("%d.%m.%Y %H:%M"))
                            
                        #print(toolboxes[cc] + ": ")
                        #print( str(timeVectors[cc]))
                        
    
        lineNumber = lineNumber + 1;

['matlab', 'simulink']
['matlab']
simulink: 16.06.2020 16:30, 16.06.2020 16:45
matlab: 16.06.2020 16:22, 16.06.2020 18:05


In [39]:
def processData(validLines, numberLines):
    lineValid = [False] * numberLines
    for idx in validLines:
        lineValid[idx] = True

    noInstances = 0      # Counts how many instances are open at the same time    
    timeMap = {}      # {ProcessID: time}
    timeVector = []
    toolboxes = []

    with open(fileName, 'r') as readFile:
        lineNumber = 0
        for line in readFile:
            # 'non valid' lines are ignored
            if lineValid[lineNumber]:

                # ------------------------------------------------------------
                # START line found:
                #     - add {processID: start time} to timeMap dict.
                # ------------------------------------------------------------
                if line.startswith('$START'):
                    line = line.rstrip('\n')  # remove \n at end of line

                    # Get process ID and time stamp
                    substr = line.split(',')  # split in substrings
                    processID = substr[1]     # get process ID
                    timeStamp = substr[4]     # get time stamp and convert to datetime format
                    timeStamp = datetime.strptime(timeStamp, "%d.%m.%Y %H:%M")

                    # Add process ID and time stamp to dict
                    timeMap[processID] = timeStamp

                    # Increase number of active instances
                    noInstances = noInstances + 1

                # -----------------------------------------------------
                # END line found:
                # If the last active instance:
                #    - Process time stamps and write output data
                # else
                #    - Add [start, end time] of process to timeVector
                #    - Add toolboxes of process to toolboxes
                #    Interpretation:
                #    timeVector[2n], timeVector[2n+1] is the start/end time
                #    of a process with toolboxes toolboxes[n].
                # -----------------------------------------------------

                elif line.startswith("$END"):

                    # Get data
                    line = line.rstrip('\n')  # remove \n at end of line

                    substr = line.split(',')  # split in substrings

                    processID = substr[1]
                    licenseNo = substr[2]
                    version   = substr[3]
                    endTime = substr[4]
                    endTime = datetime.strptime(endTime, "%d.%m.%Y %H:%M")
                    tbox = substr[5].strip('][').split(':')

                    # Add start - end time and toolboxes
                    if not processID in timeMap:
                        print("ERROR: (line number = " + str(lineNumber) + ")" )
                        print("No start time found for the end timestamp.")
                        #return
                    else:
                        startTime = timeMap[processID]
                        timeVector.append(startTime)
                        timeVector.append(endTime)

                        toolboxes.append(tbox)

                    # One instance closed by "END" => decrease active instance counter
                    noInstances = noInstances - 1

                    # ---------------------------------------------------
                    # If was last active instance: process & output data
                    # ---------------------------------------------------
                    if noInstances == 0:
                        toolboxes, timeVectors = processTime(timeVector, toolboxes)

                        for cc in range(len(toolboxes)):
                            toolbox = toolboxes[cc]
                            noIntervals = int(len(timeVectors[cc])/2)

                            for cy in range(noIntervals):
                                startTime = timeVectors[cc][2*cy]
                                endTime = timeVectors[cc][2*cy+1]

                                print(toolboxes[cc] + ": "
                                      + startTime.strftime("%d.%m.%Y %H:%M") + ", "
                                      + endTime.strftime("%d.%m.%Y %H:%M"))

                            #print(toolboxes[cc] + ": ")
                            #print( str(timeVectors[cc]))


            lineNumber = lineNumber + 1;

In [40]:
processData(validLines, numberLines)

simulink: 16.06.2020 16:30, 16.06.2020 16:45
matlab: 16.06.2020 16:22, 16.06.2020 18:05


In [28]:
####################################################################
## mergeTimeStamps()
##
## Detect and merge overlapping time stamps.
##
## Input:
##    - timeStamps: List of time stamps. Time stamp k:
##      start_time = timeStamps[2*k], end_time = timeStamps[2*k+1]
##
## Output:
##    - mergedTimeArray: List of non-overlapping time stamps.
##
## Example: 
##    timeStamps = [0, 4, 6, 9, 3, 7, 10, 11]
##    => mergedTimeArray = [0, 9, 10, 11]
##    (Works analogously if entries are of format datetime()).
####################################################################

def mergeTimeStamps( timeStamps ):
    mergedTimeArray = []   # If a non-overlapping time stamp found, it
                           # will be added to this output array.
    
    if len(timeStamps) % 2 != 0:
        print("[mergeTimeStamps] ERROR: Unexpected number of timestamp entries.")
        return;
    
    noElements = int(len(timeStamps)/2)   # Total number of timestamps
    indices = range(0, noElements)        # Indices for loop over timestamps
    activeElements = [True] * noElements  # Indicates, which timestamp elements are yet unprocessed
    
    # Select first element to be processed
    element = [timeStamps[0], timeStamps[1]]
    activeElements[0] = False              
    
    # While there are still unprocessed elements in timeStamps vector
    while any(activeElements):   
        afterMerge = False 
        
        # Check timeStamp 'element' for overlap with all other (active)
        # elements of timeStamp vector.
        for n in [idx for idx in indices if activeElements[idx]]:
            
            timePair = [timeStamps[2*n], timeStamps[2*n + 1]]
            
            # If overlap found => merge time stamps
            if not (timePair[0] > element[1] or timePair[1] < element[0]):
                element = [min(element[0], timePair[0]), max(element[1], timePair[1])]
                activeElements[n] = False
                afterMerge = True
                break            # The test will start over with the new merged element
        
        # If no overlap with any other time interval found:
        # Add element to output array, and set the next unprocessed timeStamp as the
        # element to be processed.
        if not afterMerge:
            mergedTimeArray = mergedTimeArray + element
            
            idx = activeElements.index(True)
            element = [timeStamps[2*idx], timeStamps[2*idx+1]]
    
    # All elements have been processed, add last element to output array.
    mergedTimeArray = mergedTimeArray + element
    
    return mergedTimeArray
    

In [26]:
#######################################################################
## processTime()
##
## Converts a list of time stamps and associated toolboxes
## to a list of unique time stamps for each toolbox.
##
## Input parameters:
##    - timeVector: Vector of start/end times. Time stamp k
##        is formed by:
##                  start_time(k) = timeVector[2k]
##                  end_time(k) = timeVector[2k+1]
##    - toolboxes: List of toolboxes associated with a time stamp.
##      List of toolboxes associated with time stamp k:
##      toolboxes[k]
##
## Output parameters:
##    - uniqueToolboxes: List of unique toolbox names.
##    - time_list: List of start/end times vectors. 
##      time_list[k] is the start/end time vector for 
##      toolbox uniqueToolboxes[k]. A vector can consist of multiple
##      time stamps (multiple start/end time pairs).
## 
## Example:
## Input
##       timeVector = [datetime(2020,6,16,16,30),
##                     datetime(2020,6,16,16,45),
##                     datetime(2020,6,16,16,22),
##                     datetime(2020,6,16,18,5)]
##       toolboxes = [['matlab', 'simulink'],
##                    ['matlab']]
##
## Output
##       uniqueToolboxes = ['simulink', 'matlab']
##       time_list = [[datetime(2020,6,16,16,30), datetime(2020,6,16,16,45)],
##                    [datetime(2020,6,16,16,22), datetime(2020,6,16,18,5)]]
#############################################################################


def processTime(timeVector, toolboxes):
    
    time_list = []
    
    # Determine unique toolbox names
    tboxesFlat = [item for sublist in toolboxes for item in sublist]
    uniqueToolboxes = list(set(tboxesFlat)) 
    
    # For each toolbox: Create an associated time stamp vector.
    for toolbox in uniqueToolboxes:
        timeTbx = []    
             
        cc = 0
        
        # Loop over all time stamps 
        # (specifically: list of toolboxes for each time stamp)
        for tbxList in toolboxes:
            # If the time stamp included the current toolbox:
            # add time stamp to toolbox time vector.
            if toolbox in tbxList:
                timeTbx.append(timeVector[2*cc])   #start time
                timeTbx.append(timeVector[2*cc+1]) #end time
            cc = cc + 1;
        
        # Detect and merge overlapping time stamps
        timeTbx = mergeTimeStamps(timeTbx)
        
        # Add to output vector
        time_list.append(timeTbx)
        
    return uniqueToolboxes, time_list

In [7]:
tlist = [0, 4, 6, 9, 3, 7, 10, 11]
mergeTimeStamps(tlist)

[0, 9, 10, 11]

In [21]:
tlist = [datetime(2020, 6, 16, 16, 30),
        datetime(2020,6,16,16,45),
        datetime(2020,6,16,16,22),
        datetime(2020,6,16,18,5)]
mergeTimeStamps(tlist)

[datetime.datetime(2020, 6, 16, 16, 22), datetime.datetime(2020, 6, 16, 18, 5)]

In [8]:
tboxes, time_list = processTime(timeVector, toolboxes)
tboxes
for tbx in tboxes:
    print(tbx)

NameError: name 'timeVector' is not defined

In [137]:
isActive = [True]*6
isActive[2] = False
indices = range(0,6)

res = [idx for idx in indices if isActive[idx]]
print(res)

for a in [1,3,5]:
    print(a)

[0, 1, 3, 4, 5]
1
3
5


In [67]:
myString = '[matlab,simulink]'
#myString = "[1, 2, 3, 4, 5]"
res = myString.strip('][').split(',')   # use .split(', ') if additional space
res

['matlab', 'simulink']

In [72]:
myString = "$START,13828,40913439,27 (R2020) Update 1,16.06.2020 16:22"
dateString = myString.split(',')[4]
display(dateString)

'16.06.2020 16:22'

In [89]:
time0 = datetime.strptime(dateString, "%d.%m.%Y %H:%M")
time1 = datetime(2020, 6, 16, 17, 30)

dt = time1 - time0
display(dt.total_seconds()/(60*60))  

time1.strftime("%d.%m.%Y %H:%M")

1.1333333333333333

'16.06.2020 17:30'

In [151]:
isActive = [False] * 8
isActive[2] = False
isActive[4] = True

if any(isActive):
    print("yes")
    
isActive.index(True)

yes


4

In [30]:
class LogFileProcessor:
    
    def __init__(self, name):
        self.name = name
        
    def print(self):
        print("Name = " + self.name)
        
    

In [31]:
logFileProcessor = LogFileProcessor("Janice")

In [32]:
logFileProcessor.print()

Name = Janice
